In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Load the dataset
file_path = "heart.csv"
df = pd.read_csv(file_path)

# Encode categorical variables
categorical_cols = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']
df = pd.get_dummies(df, columns=categorical_cols)

# Split data into features and target
X = df.drop("HeartDisease", axis=1)
y = df["HeartDisease"]

# Normalize numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply SMOTE for data augmentation
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train an SVM model
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Function to classify new patient data
def classify_patient(data):
    data_df = pd.DataFrame([data], columns=X.columns)
    data_scaled = scaler.transform(data_df)
    prediction = svm_model.predict(data_scaled)
    probability = svm_model.predict_proba(data_scaled)[0]
    return "Heart Disease" if prediction[0] == 1 else "No Heart Disease", probability

# Take user input
print("Enter the following details:")
user_data = {}
for col in X.columns:
    user_data[col] = float(input(f"{col}: "))

prediction, probability = classify_patient(user_data)
print(f"Predicted Diagnosis: {prediction}")
print(f"Confidence: {max(probability) * 100:.2f}%")


Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.80      0.81        88
           1       0.85      0.87      0.86       116

    accuracy                           0.84       204
   macro avg       0.84      0.83      0.83       204
weighted avg       0.84      0.84      0.84       204

Enter the following details:
Age: 96
RestingBP: 134
Cholesterol: 189
FastingBS: 123
MaxHR: 100
Oldpeak: 2
Sex_F: 0
Sex_M: 1
ChestPainType_ASY: 2
ChestPainType_ATA: 3
ChestPainType_NAP: 1
ChestPainType_TA: 2
RestingECG_LVH: 4
RestingECG_Normal: 2
RestingECG_ST: 3
ExerciseAngina_N: 1
ExerciseAngina_Y: 4
ST_Slope_Down: 1
ST_Slope_Flat: 4
ST_Slope_Up: 1
Predicted Diagnosis: Heart Disease
Confidence: 100.00%
